In [11]:
from finml.finml.data.handler import StandardNorm
import pickle
import numpy as np
import pandas as pd
import polars as pl

In [ ]:
scaler = StandardNorm()

In [ ]:
scaler.fitted = True

In [ ]:
scaler.is_fitted()

In [ ]:
with open("scaler.pkl","wb") as f:
    pickle.dump(scaler,f)

In [ ]:
with open("scaler.pkl","rb") as f:
    scaler2= pickle.load(f)

In [ ]:
scaler2.is_fitted()

In [ ]:
#scaler.fit_transform([1,2,3,4,5],[2,3,4,5,6])

In [14]:
df = pl.read_parquet("example_data/ETHBUSD-PERP.BINANCE-250-VALUE_IMBALANCE-LAST-EXTERNAL.parquet")

In [15]:
from features.features import OrderBlockDetecter

In [43]:
detecter = OrderBlockDetecter(5,0.001,3)

In [44]:
order_blocks_times = [] 
order_blocks_values = [] 
is_bulls = [] 
for i in range(2000):
    detecter.update_raw(df[i,"high"],df[i,"low"],df[i,"close"],df[i,"volume"],int(df[i,"datetime"]))
    if detecter.initialized():
        if detecter.bull_blocks_num()>=1:
            bull_info = detecter.get_bull_block(0)
            if len(order_blocks_times)< 1 or bull_info[-1]!=order_blocks_times[-1]:
                order_blocks_times.append(bull_info[-1])
                order_blocks_values.append(bull_info[-2])
                is_bulls.append(True)
        if detecter.bear_blocks_num()>=1:
            bear_info = detecter.get_bear_block(0)
            if len(order_blocks_times)<1 or bear_info[-1]!=order_blocks_times[-1]:
                order_blocks_times.append(bear_info[-1])
                order_blocks_values.append(bear_info[-2])
                is_bulls.append(False)

In [45]:
import plotly.express as px

In [46]:
sub_df = df[:2000]

In [47]:
df2 = pd.DataFrame({"datetime":order_blocks_times,
                   "values":order_blocks_values,
                   "is_bull":is_bulls})

In [48]:
import plotly.graph_objects as go
# create an empty figure
fig = go.Figure()

In [49]:
df3 = df2[df2["is_bull"]]

In [54]:
import plotly.graph_objects as go
# create an empty figure
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[detecter.get_bear_block(0)[-1],detecter.get_bear_block(1)[-1]],
        y=[detecter.get_bear_block(0)[-2],detecter.get_bear_block(1)[-2]])
)
fig.add_trace(
    go.Line(
        
        x=sub_df["datetime"],
        y=sub_df["close"]
    )
)
fig.show()

/home/clark/.cache/pypoetry/virtualenvs/nautilus-trader-hztl02LS-py3.10/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [38]:
df3

,datetime,values,is_bull
0,1643922034976000000,2635.11,False
12,1644049608124999936,3037.00,False
14,1644049608124999936,3037.00,False
16,1644049608124999936,3037.00,False
18,1644049608124999936,3037.00,False
...,...,...,...
955,1645467448379000064,2704.56,False
957,1645467448379000064,2704.56,False
959,1645467448379000064,2704.56,False
961,1645467448379000064,2704.56,False
